In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from gensim.test.utils import common_texts, get_tmpfile
from sklearn.model_selection import GridSearchCV
from gensim.models import Word2Vec
from standard_function import *
from return_detail_icd_code import *
import pickle
import unicodedata
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

# Load data, Models and wb File

In [2]:
load_data_check_default = pd.read_excel("ICD_new_sp.xlsx", header = None)
df = pd.read_excel("ICD_new.xlsx", header = None, encoding = 'utf-8')

load_data_check = load_data_check_default.copy()
load_data_check[1] = load_data_check[1].apply(delete_special_char)
#load_data_check[1] = load_data_check[1].apply(un_punctuation)

In [4]:
f = open('Eclaim_predict.pickle', 'rb')      # load model dự đoán mã icd 
classifier = pickle.load(f)
f.close()

In [5]:
def try_except_2gram ( string):               # Load hàm và xử lý ngoại lệ cắt từ
    model = Word2Vec.load("2_gram.model")
    try :  
        model.wv.similar_by_word(string)
    except :
        kq = False
    else :
        kq = True
    return kq

In [6]:
def try_except_3gram ( string):               # Load hàm và xử lý ngoại lệ sửa từ
    model = Word2Vec.load("3_gram.model")
    try :  
        model.wv.similar_by_word(string)
    except :
        kq = False
    else :
        kq = True
    return kq

In [ ]:
with open ('exacly_word', 'rb') as fp:       # Load Sample Correct words
    words = pickle.load(fp)

# Xử lý đầu vào Input

In [7]:
#text = input("Cung cấp chuỗi cần xác định :")
text = 'VIÊM GAN MẠN / NHIỄM TRÙNG MU BÀN CHÂN (P)'

text = out_special_char(text)      # hàm loại bỏ dấu ngoặc tròn, vuông , \ , /
pr = spaces_num(char_num(text))    # loại bỏ khoản trắng vô nghĩa  
pr1 = ' '.join(pr)                 # convert  list to string
pr1 = delete_special_char(pr1)     # delete others special char --- > string return string

In [8]:
text1 = pr1.strip("").split(" ")       # Convert str to list
text1 = spaces_num(text1)                # loại bỏ khoản trắng thừa trong list
text2 = pr1.strip("").split("*")       # Convert str to list Text 1 chứa list các từ đơn
text2 = spaces_num(text2)                # Text2 chứa List cả cụm từ
print(text1)
print(text2)

['viêm', 'gan', 'mạn', 'nhiễm', 'trùng', 'mu', 'bàn', 'chân', 'p']
['viêm gan mạn nhiễm trùng mu bàn chân p']


In [17]:
new_str =''
for i in range(0,len(text1)-1):
    new_str += text1[i] + ' ' + text1[i+1] + ","
new_str

'viêm gan,gan mạn,mạn nhiễm,nhiễm trùng,trùng mu,mu bàn,bàn chân,chân p,'

In [10]:
new_str = new_str.split(",")
addr = []
chuoi = ''
for index,i in enumerate (new_str[:len(new_str)-2]):
    if try_except_2gram(i)== False :
        addr.append(index)
        chuoi += i + ","
chuoi

'mạn nhiễm,trùng mu,'

In [11]:
check = ''
tg = 0
if len(addr) ==0:
    check = text2
else:
    for index, value in enumerate(text1):
        for index2 in addr:
            if index == (index2+1):
                check += ','
        check += text1[index] + " "
check = check.strip(' ').split(',')
check

['viêm gan mạn ', 'nhiễm trùng ', 'mu bàn chân p']

In [12]:
with open ('sample_vector', 'rb') as fp:
    itemlist = pickle.load(fp)

In [13]:
count = CountVectorizer()
count.fit(itemlist)
bag_of_words = count.transform(itemlist)

In [14]:
x_new1 = count.transform(check)
x_new1
kq = classifier.predict(x_new1)

In [15]:
res = ''
for g in range(0,len(kq)):
    df_end = (load_data_check[load_data_check[2] == kq[g]])
    io = decission_value_return_01(df_end, check[g].split(','))
    ii = load_data_check_default[load_data_check_default[0]== io[0]].index.item()
    res += str(io[0]) + ':' + str(io[1]) + ':' + str(io[2]) + ':' +load_data_check_default[1][ii] + '**'
op = spaces_num(res.strip('').split('**'))

op

['B18:1.0:6:Viêm gan mạn',
 'A41:1.0:9:Nhiễm trùng khác',
 'Q66:0.6666666666666666:11:Các biến dạng bẩm sinh của bàn chân']

In [16]:
res1=''
for g in range(0,len(kq)):
    df_end = (load_data_check[load_data_check[2] == kq[g]])
    io = decission_value_return_02(df_end, check[g].split(','))
    ii = load_data_check_default[load_data_check_default[0]== io[0]].index.item()
    res1 += str(io[0]) + ':' + str(io[1]) + ':' + str(io[2]) + ':' +load_data_check_default[1][ii] + '**'
op = spaces_num(res1.strip('').split('**'))
op

['B18:1.0:2:Viêm gan mạn',
 'A41:0.6666666666666666:1:Nhiễm trùng khác',
 'Q66:0.6666666666666666:2:Các biến dạng bẩm sinh của bàn chân']